brukau 10:13 AM
morning @Joris Mattheijssens
so here are there first endpoints

https://peach.ebu.io/api/v1/bevrt/cpn-als?user_id=30386775984251830562577782988368845581

https://peach.ebu.io/api/v1/bevrt/cpn-hybrid?user_id=30386775984251830562577782988368845581

https://peach.ebu.io/api/v1/bevrt/cpn-random-recent?user_id=30386775984251830562577782988368845581

first one is basic als which we are going to use as baseline for evaluation

cpn-hybrid is als based but mixed with recency rescoring and similar content (edited) 

random-recent is just randomly chosen from recently published (also to compare in the avaluation)

I am running now the model calculation last time based on the data from Aug, 1 to Sept5 including and then it will be "fixed"

In [138]:
import numpy as np
import datetime
import pandas as pd
from collections import Counter
import re
import boto3
import os
import json
import operator
import matplotlib.pyplot as plt
import tarfile
import shutil
import urllib.request


In [139]:
fileCred = os.getenv("HOME") + '/.aws/credentials-s3'
fileCred = os.getenv("HOME") + '/.aws/credentials'

bucketIn = 'vrt-adobe-analytics'
folderIn = 'datafeed'

bucketArticles = 'vrtnws-articles2'
fileCatalogue = 'catalogue.csv.gz'



# Load

In [140]:
# Read credentials
with open(fileCred, 'r') as f:
    for line in f:
        if not re.match('aws_access_key_id', line) is None:
            ACCESS_KEY = line.split('=')[1].strip()
        elif not re.match('aws_secret_access_key', line) is None:
            SECRET_KEY = line.split('=')[1].strip()

session = boto3.Session(aws_access_key_id=ACCESS_KEY,
                        aws_secret_access_key=SECRET_KEY)
s3 = session.resource('s3')  

os.environ['AWS_ACCESS_KEY_ID'] = ACCESS_KEY
os.environ['AWS_SECRET_ACCESS_KEY'] = SECRET_KEY
s3 = boto3.resource('s3')
my_bucket = s3.Bucket(bucketIn)  

In [150]:
# List of lookup data files
fileLookup = []
for s3_file in my_bucket.objects.all():
    if not re.search(folderIn + '/vrtproduction_.*.tar.gz', s3_file.key) is None:
        fileLookup.append(s3_file.key)
    
fileLookup.sort(reverse=True)

In [148]:
# List of data files
fileData = []
for s3_file in my_bucket.objects.all():
    if not re.search(folderIn + '/01-vrtproduction_.*.tsv.gz', s3_file.key) is None:
        fileData.append(s3_file.key)
    
fileData.sort(reverse=True)

In [151]:
f = 1


# Load headers
print(fileLookup[f])
my_bucket.download_file(fileLookup[f], 'tmp.tar.gz')

tar = tarfile.open("tmp.tar.gz")
tar.extractall('lookup')
tar.close()

file = open('lookup/column_headers.tsv', 'r')
headers = file.readline().split('\t')
file.close()

shutil.rmtree('lookup')
os.remove('tmp.tar.gz')



# Load events
print(fileData[f])
my_bucket.download_file(fileData[f], 'tmp.tsv.gz')
events_ = pd.read_csv('tmp.tsv.gz', sep='\t', delimiter=None, header=None, names=None, index_col=None, 
           usecols=None, squeeze=False, prefix=None, mangle_dupe_cols=True, dtype=None, engine=None, 
           converters=None, true_values=None, false_values=None, skipinitialspace=False, 
           skiprows=None, nrows=None, na_values=None, keep_default_na=True, na_filter=True, 
           verbose=False, skip_blank_lines=True, parse_dates=False, infer_datetime_format=False, 
           keep_date_col=False, date_parser=None, dayfirst=False, iterator=False, chunksize=None, 
           compression='gzip', thousands=None, decimal=b'.', lineterminator=None, quotechar='"', 
           quoting=0, escapechar=None, comment=None, encoding='latin_1', dialect=None, 
           tupleize_cols=None, error_bad_lines=False, warn_bad_lines=True, skipfooter=0, 
           doublequote=True, delim_whitespace=False, low_memory=False, memory_map=False, 
           float_precision=None)
events_.columns = headers
events_ = events_[events_.post_pagename.str.contains('vrtnws', na=False, regex=True)].reset_index(drop=True)

os.remove('tmp.tsv.gz')


# events_

datafeed/vrtproduction_20190906-090000-lookup_data.tar.gz
datafeed/01-vrtproduction_20190906-090000.tsv.gz


In [163]:
events_

,post_pagename,post_prop6,post_page_url,post_evar4,post_evar5,post_evar3,post_evar27,post_evar7,post_evar8,post_evar9,...,post_videocontenttype,post_evar31,post_evar34,post_evar38,post_evar14,post_visid_high,post_visid_low,post_visid_type,hit_time_gmt,post_prop15
0,/vrtnws/nl/2019/09/05/_brexit-moeheid-bij-vlaa...,/vrtnws/nl/2019/09/05/_brexit-moeheid-bij-vlaa...,https://www.vrt.be/vrtnws/nl/2019/09/05/_brexi...,web,vrt nws,nl,1567691587919,NaN,binnenland,brexit,...,NaN,NaN,NaN,NaN,NaN,85456735,3429059808,1,1567754367,"binnenland,brexit,charlotte-van-driessche"
1,/vrtnws/nl/net-binnen/,/vrtnws/nl/net-binnen/,https://www.vrt.be/vrtnws/nl/net-binnen,web,vrt nws,nl,1509016329578,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,632902389,583480437,1,1567753915,NaN
2,/vrtnws/nl/2019/09/06/26-jarige-belg-al-10-dag...,/vrtnws/nl/2019/09/06/26-jarige-belg-al-10-dag...,https://www.vrt.be/vrtnws/nl/2019/09/06/26-jar...,web,vrt nws,nl,1567742033599,NaN,buitenland,NaN,...,NaN,NaN,NaN,NaN,NaN,632902389,583480437,1,1567753995,"buitenland,frederik-defossez"
3,/vrtnws/nl/2019/09/05/schelde-werd-plots-troeb...,/vrtnws/nl/2019/09/05/schelde-werd-plots-troeb...,https://www.vrt.be/vrtnws/nl/2019/09/05/scheld...,web,vrt nws,nl,1567716352439,NaN,wetenschap,NaN,...,NaN,NaN,NaN,NaN,NaN,632902389,583480437,1,1567754048,"luc-de-roy,wetenschap"
4,/vrtnws/nl/2019/09/05/brandweer-elektrische-wa...,/vrtnws/nl/2019/09/05/brandweer-elektrische-wa...,https://www.vrt.be/vrtnws/nl/2019/09/05/brandw...,web,vrt nws,nl,1567706280301,NaN,binnenland,NaN,...,NaN,NaN,NaN,NaN,NaN,632902389,583480437,1,1567754296,"opinie,binnenland"
5,/vrtnws/nl/2019/09/05/frank-vandenbroucke-verg...,/vrtnws/nl/2019/09/05/frank-vandenbroucke-verg...,https://www.vrt.be/vrtnws/nl/2019/09/05/frank-...,web,vrt nws,nl,1567709949474,frankvdb,politiek,kies19,...,NaN,frankvdb,NaN,NaN,NaN,632902389,583480437,1,1567754436,"video,rik-arnoudt,politiek,kies19,federaal-par..."
6,/vrtnws/nl/2019/09/05/_brexit-moeheid-bij-vlaa...,/vrtnws/nl/2019/09/05/_brexit-moeheid-bij-vlaa...,https://www.vrt.be/vrtnws/nl/2019/09/05/_brexi...,web,vrt nws,nl,1567691587919,NaN,binnenland,brexit,...,NaN,NaN,NaN,NaN,NaN,632902389,583480437,1,1567754535,"binnenland,brexit,charlotte-van-driessche"
7,/vrtnws/nl/2019/09/05/schelde-werd-plots-troeb...,/vrtnws/nl/2019/09/05/schelde-werd-plots-troeb...,https://www.vrt.be/vrtnws/nl/2019/09/05/scheld...,web,vrt nws,nl,1567716352439,NaN,wetenschap,NaN,...,NaN,NaN,NaN,NaN,NaN,632902389,583480437,1,1567754640,"luc-de-roy,wetenschap"
8,/vrtnws/nl/2019/09/06/26-jarige-belg-al-10-dag...,/vrtnws/nl/2019/09/06/26-jarige-belg-al-10-dag...,https://www.vrt.be/vrtnws/nl/2019/09/06/26-jar...,web,vrt nws,nl,1567742033599,NaN,buitenland,NaN,...,NaN,NaN,NaN,NaN,NaN,632902389,583480437,1,1567754642,"buitenland,frederik-defossez"
9,/vrtnws/nl/net-binnen/,/vrtnws/nl/net-binnen/,https://www.vrt.be/vrtnws/nl/net-binnen,web,vrt nws,nl,1509016329578,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,632902389,583480437,1,1567754647,NaN


In [165]:
url = 'https://peach.ebu.io/api/v1/bevrt/cpn-als?user_id=30386775984251830562577782988368845581'
page = urllib.request.urlopen(url)
print(page.read())

b'{"status": "ok", "codops": "bevrt", "result": {"items": [["1565348163426", 0.007604820188134909], ["1565719782509", 0.0075384280644357204], ["1566061132213", 0.0067713381722569466], ["1565939553641", 0.006522053852677345], ["1565621283532", 0.005478191189467907], ["1565940111249", 0.005277711432427168], ["1565754390109", 0.005128633696585894], ["1565612856885", 0.004758703522384167], ["1565773413642", 0.004643669351935387], ["1566987844764", 0.00452735461294651], ["1565897865896", 0.004226141143590212], ["1565794762199", 0.003928499296307564], ["1565803075543", 0.003920349292457104], ["1565895103355", 0.0038672511000186205], ["1565768330155", 0.0038280629087239504], ["1565934876383", 0.0036844697315245867], ["1565850702745", 0.003599934745579958], ["1565275375321", 0.0035504139959812164], ["1566968526250", 0.003394327824935317], ["1562595555079", 0.003324156394228339], ["1562865337307", 0.0033028735779225826], ["1562601589529", 0.0032808633986860514], ["1565867883429", 0.003203546162

In [166]:
for i in np.random.choice(range(len(events_)), 100, replace=False):
    user_id = str(events_.post_visid_high[i]) + str(events_.post_visid_low[i])
    url = 'https://peach.ebu.io/api/v1/bevrt/cpn-als?user_id=' + user_id
    page = urllib.request.urlopen(url)
    if (page.read() != b'{"status": "ok", "message": "algorithm did not return any data", "fallback_used": false, "bevrt": "up"}'):
        print(user_id)
        
    user_id = str(events_.post_visid_low[i]) + str(events_.post_visid_high[i]) 
    url = 'https://peach.ebu.io/api/v1/bevrt/cpn-als?user_id=' + user_id
    page = urllib.request.urlopen(url)
    if (page.read() != b'{"status": "ok", "message": "algorithm did not return any data", "fallback_used": false, "bevrt": "up"}'):
        print(user_id)
